In [4]:
%run efx_charity

loaded efx algorithm


## Example

In [5]:
#n, t, assignments, items, valuations
data = (3,3,
 np.zeros((3,3)),
 np.array([10., 30., 10.]),
 np.array([[4.2, 2., 1],
        [3., 2.5, 0.1],
        [1., 5., 29]]))
run_and_check(draw=True, inputs=data)

APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
DE-CYCLING GRAPH
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
COULD NOT APPLY U0
trying U2 with source: 0
APPLIED U2
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
APPLIED U0
COULD NOT APPLY U0
trying U2 with source: 1
APPLIED U2
APPLIED U0
APPLIED U0
APPLIED U0
COULD NOT APPLY U0
trying U2 with source: 2
trying U2 with source: 0
trying U2 with source: 1
COULD NOT APPLY U2 (!)
player 0 assignments: [ 3. 10.  4.]
player 1 assignments: [4. 9. 0.]
player 2 assignments: [ 2. 11.  6.]


AssertionError: ALL ITEMS SHOULD BE ALLOCATED

## Randomly generate cases

In [8]:
n = 4
t = 3
items = [10, 10, 10]
for i in range(500):
    if i % 100 == 0:
        print(i)
    valuations = np.random.random((n,t))*10
    data = (n,t, np.zeros((n,t)), items, valuations)
    out = run(inputs=data, log=False)
    
    while not check_all_done(out[2], out[3]):
        print('i = ', i, 'trying again...')
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr(inputs=data)  # "ALL ITEMS SHOULD BE ALLOCATED"


0
COULD NOT APPLY U2 (!)
i =  15 trying again...


NameError: name 'run_rr' is not defined

## Another algorithm

This one does round robin on any source.

In [10]:
def run_rr(inputs=None, log=False):
    pr = print if log else lambda *x: None
    n, t, assignments, items, valuations = inputs if inputs else setup()
    # algorithm
    while True:
        # assert partial EFX
#         assert check_all_efx(assignments, valuations), "this should be partial EFX"
        # de cycle graph if needed
        G = create_envy_digraph(assignments, valuations, n)
        is_cycle, cycle = locate_envy_cycle(G)
        while is_cycle:
            pr("DE-CYCLING GRAPH")
            cycle_mask = cycle
            cycle_mask_new = cycle_mask[1:] + cycle_mask[0:1]
            assignments[cycle_mask_new] = assignments[cycle_mask]
            G = create_envy_digraph(assignments, valuations, n)
            is_cycle, cycle = locate_envy_cycle(G)
        assert is_directed_acyclic_graph(G)
        # check if done
        if check_all_done(assignments, items):
            pr("DONE")
            return n, t, assignments, items, valuations
        # find a source and assign it its favorite item
        all_sources = [s for s,d in G.in_degree() if d == 0]
        np.random.shuffle(all_sources)
        added_item = False
        for s in all_sources:
            prefs = np.argsort(valuations[s,:])[::-1]
            for p in prefs:
                if item_type_available(assignments, items, p):
                    # if available but not EFX, dont use this source
                    assignments[s,p] += 1
                    if check_all_efx(assignments, valuations):
                        pr("assigning type", p, "to player", s)
                        added_item = True
                    else:
#                         print('skipping type', p, 'for ', s)
                        assignments[s,p] -= 1
                    break
            if added_item:
                break
        if added_item:
            continue
        pr("COULDNT ADD ITEM(?)")
        return n, t, assignments, items, valuations

In [23]:
# look for tiny dead end
n = 4
t = 4
trials = 2000
all_items = np.random.randint(2, 5, (trials, t)) # start with 10 items each to reduce randomness
all_valuations = np.random.randint(1, 50, (trials, n,t))

for i in range(trials):
    if i % 100 == 0:
        print(i)
    items = all_items[i, :]
    valuations = all_valuations[i, :, :]
    data = (n,t, np.zeros((n,t)), items, valuations)
    
    out = run_rr(inputs=data)
    assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
    while not check_all_done(out[2], out[3]):
        print('i = ', i, 'trying again...')
        data = (n,t, np.zeros((n,t)), items, valuations)
        raise 
        out = run_rr(inputs=data)  # "ALL ITEMS SHOULD BE ALLOCATED"

0
i =  65 trying again...


RuntimeError: No active exception to reraise

In [24]:
out

(4,
 4,
 array([[1., 1., 2., 0.],
        [3., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.]]),
 array([4, 2, 3, 2]),
 array([[24, 18, 35, 36],
        [ 6, 11,  4, 35],
        [10,  3, 13, 38],
        [ 3,  5,  1, 44]]))

In [34]:
run_rr((n,t, np.zeros((n,t)), items, valuations), log=True)

assigning type 3 to player 1
assigning type 3 to player 3
assigning type 2 to player 0
assigning type 2 to player 2
assigning type 2 to player 0
assigning type 0 to player 2
assigning type 0 to player 2
assigning type 0 to player 0
assigning type 0 to player 2
assigning type 1 to player 3
assigning type 1 to player 1
DONE


(4,
 4,
 array([[1., 0., 2., 0.],
        [0., 1., 0., 1.],
        [3., 0., 1., 0.],
        [0., 1., 0., 1.]]),
 array([4, 2, 3, 2]),
 array([[24, 18, 35, 36],
        [ 6, 11,  4, 35],
        [10,  3, 13, 38],
        [ 3,  5,  1, 44]]))

In [17]:
run_rr((n,t, np.zeros((n,t)), items, valuations), log=True)

assigning type 1 to player 0
assigning type 0 to player 1
assigning type 0 to player 1
assigning type 2 to player 0
DONE


(2,
 3,
 array([[0., 1., 1.],
        [2., 0., 0.]]),
 [2, 1, 1],
 array([[1, 3, 1],
        [3, 4, 1]]))

In [798]:
trials = 1
successes = np.zeros(trials)

repeats = 10000
for i in range(trials):
    wins = 0
    for _ in range(repeats):
        if _ % 1000 == 0:
            print(_)
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr(inputs=data)
        assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
        wins += check_all_done(out[2], out[3])
    successes[i] = wins/repeats

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [895]:
successes

array([0.5034])

what about stacking them together?


In [1093]:
def make_out(k):
    make_zeros = lambda: np.zeros((2*k,3*k))
    repeater = [[1, 1, 4],
    [3, 1, 4]]

    test = make_zeros()
    for i in range(k):
        test[2*i:2*i+2, 3*i:3*i+3] = repeater
    return (2*k,
 3*k,
 make_zeros(),
 [2, 1, 1]*k,
       test)

In [1095]:
run_rr(make_out(2))

(4,
 6,
 array([[0., 1., 1., 1., 0., 0.],
        [2., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0.],
        [0., 0., 0., 0., 0., 1.]]),
 [2, 1, 1, 2, 1, 1],
 array([[1., 1., 4., 0., 0., 0.],
        [3., 1., 4., 0., 0., 0.],
        [0., 0., 0., 1., 1., 4.],
        [0., 0., 0., 3., 1., 4.]]))

In [1097]:
trials = 1
successes = np.zeros(trials)

repeats = 1000
for i in range(trials):
    wins = 0
    for _ in range(repeats):
        if _ % 100 == 0:
            print(_)
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr(inputs=make_out(2))
        assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
        wins += check_all_done(out[2], out[3])
    successes[i] = wins/repeats
successes

0
100
200
300
400
500
600
700
800
900


array([0.738])

new counterexample

In [358]:
valuations = np.array([
    [1, 1.5, 0],
    [4, 1.5, 1],
    [0, 1.5, 1]
])

In [359]:

for i in range(1):
    # assignments = np.array([[3,5,0], [6,3,0], [0,0,11]])
    data = (n,t, np.zeros((n,t)), items, valuations)

    out = run_rr(inputs=data)
    assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
    while not check_all_done(out[2], out[3]):
        print(out[2])
        print('i = ', i, 'trying again...')
        break
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr(inputs=data)  # "ALL ITEMS SHOULD BE ALLOCATED"


ValueError: shapes (4,5) and (3,) not aligned: 5 (dim 1) != 3 (dim 0)

counterexample from self mea

In [43]:
n = 3
t = 3
items = [9,9,11]
eps = 0.5
valuations = np.array([
    [1,2-eps, 0],
    [(1+eps)/(1-eps), 1+eps, 1],
    [0, 2-eps, 1]
])

for i in range(100):
#     assignments = np.array([[3,5,0], [6,3,0], [0,0,11]])
    data = (n,t, np.zeros((n,t)), items, valuations)

    out = run(inputs=data, log=False)
    assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
    if not check_all_done(out[2], out[3]):
        print(out[2])
        print('i = ', i, 'fails...')
#         data = (n,t, np.zeros((n,t)), items, valuations)
#         out = run(inputs=data, log=False)  # "ALL ITEMS SHOULD BE ALLOCATED"


COULD NOT APPLY U2 (!)
[[3. 3. 4.]
 [4. 2. 3.]
 [2. 3. 4.]]
i =  63 fails...
COULD NOT APPLY U2 (!)
[[4. 2. 4.]
 [4. 2. 5.]
 [1. 4. 2.]]
i =  80 fails...


### Look at chance of success

In [1172]:
n = 4
t = 4
trials = 200
repeats = 10
all_items = np.random.randint(9, 10, (trials, t)) # start with 10 items each to reduce randomness
all_valuations = np.random.random((trials, n,t))*10
successes = np.zeros(trials)

for i in range(trials):
    if i % 10 == 0:
        print(i)
    items = all_items[i, :]
    valuations = all_valuations[i, :, :]
    
    wins = 0
    for _ in range(repeats):
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr(inputs=data)
        assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
        wins += check_all_done(out[2], out[3])
    successes[i] = wins/repeats

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [1173]:
successes

array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.9, 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 0.9, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.9, 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.9, 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 0.8, 1. , 1. , 1. , 1. , 1. , 0.9, 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.9, 1. , 0.9, 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0.8, 1. , 1. , 1. , 1. ,
       1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 0.7, 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1.

In [1162]:
np.where(successes != 1)

(array([ 26,  51,  61,  69,  73,  74,  76, 114, 130, 147, 154, 182]),)

In [ ]:
any_fails = all_valuations[np.where(successes != 1), :, :]
any_fails

In [23]:
loser = any_fails[0][2]

In [24]:
for _ in range(3):
    n = 4
    t = 3
    trials = 5
    repeats = 100
    tests = np.zeros(trials)
    all_offsets = np.random.random((trials, n, t))*0.01
    for i in range(trials):
        print(i)
        wins = 0
        for _ in range(repeats):
            items = [10]*len(loser[0])
            valuations = loser + all_offsets[i, :, :]
            data = (n,t, np.zeros((n,t)), items, valuations)
            out = run_rr(inputs=data)
            assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
            wins += check_all_done(out[2], out[3])
        tests[i] = wins/repeats
    print(tests)
    loser = loser + all_offsets[np.argmin(tests), :, :]

0
1
2
3
4
[0.66 0.7  0.66 0.63 0.71]
0
1
2
3
4
[0.68 0.72 0.71 0.62 0.68]
0
1
2
3
4
[0.6  0.75 0.7  0.67 0.67]


In [1088]:
from scipy.optimize import minimize

In [1153]:
def test(inp, repeats=1000):
    n = 2
    t = 3
    valuations = inp.reshape((n, t))
 
    wins = 0
    for _ in range(repeats):
        items = [2, 1, 1] #hardcoded in
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr(inputs=data)
        assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
        wins += check_all_done(out[2], out[3])
    return wins/repeats

In [1157]:
test(np.array([1.01875, 1.01875, 4.125  , 3.05625, 0.925  , 4.075  ]))

0.488

In [1121]:
new_loser = make_out(1)[4].reshape(-1)

In [1159]:
minimize(test,x0=new_loser.reshape(-1), method='Powell', options={'maxiter': 2})

   direc: array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])
     fun: 0.479
 message: 'Optimization terminated successfully.'
    nfev: 89
     nit: 1
  status: 0
 success: True
       x: array([ 0.45384999,  1.381966  ,  3.00000002,  2.98328374, -0.67219124,
        4.84825532])

In [126]:
new_loser =np.array( [0.22836632, 5.25399287, 5.8437487 , 7.91702737, 7.13756416,
       8.71194301, 3.25935665, 2.87719414, 9.99483373, 7.85842655,
       3.7849645 , 4.7564503 ])

In [4]:
# most of the slowdown is checking EFX
# could 2-3x the speedup by improving this

# %prun run_rr(inputs=data)

# what about even better?

let's give items to sources that aren't sinks first

In [389]:
def run_rr2(inputs=None, random=False):
    n, t, assignments, items, valuations = inputs if inputs else setup()
    # algorithm
    while True:
        # assert partial EFX
#         assert check_all_efx(assignments, valuations), "this should be partial EFX"
        # de cycle graph if needed
        G = create_envy_digraph(assignments, valuations, n)
        is_cycle, cycle = locate_envy_cycle(G)
        while is_cycle:
#             print("DE-CYCLING GRAPH")
            cycle_mask = cycle
            cycle_mask_new = cycle_mask[1:] + cycle_mask[0:1]
            assignments[cycle_mask_new] = assignments[cycle_mask]
            G = create_envy_digraph(assignments, valuations, n)
            is_cycle, cycle = locate_envy_cycle(G)
        assert is_directed_acyclic_graph(G)
        # check if done
        if check_all_done(assignments, items):
#             print("DONE")
            return n, t, assignments, items, valuations
        # find a source and assign it its favorite item
        all_sources = [s for s,d in G.in_degree() if d == 0]
        all_sinks = [s for s,d in G.out_degree() if d == 0]
        source_not_sink = list(set(all_sources) - set(all_sinks))
        source_and_sink = list(set(all_sources).intersection(all_sinks))
        if random:
            print('randomizing')
            np.random.shuffle(source_not_sink)
            np.random.shuffle(source_and_sink)
        sources = source_not_sink + source_and_sink
#         np.random.shuffle(all_sources)
#         print(all_sources)
        added_item = False
        for s in sources:
            prefs = np.argsort(valuations[s,:])[::-1]
            for p in prefs:
                if item_type_available(assignments, items, p):
                    # if available but not EFX, dont use this source
                    assignments[s,p] += 1
                    if check_all_efx(assignments, valuations):
#                         print("assigning type", p, "to player", s)
                        added_item = True
                    else:
                        assignments[s,p] -= 1
                    break
            if added_item:
                break
        if added_item:
            continue
#         print("COULDNT ADD ITEM(?)")
        return n, t, assignments, items, valuations

In [371]:
n = 3
t = 3
trials = 1000
all_items = np.random.randint(10, 11, (trials, t)) # start with 10 items each to reduce randomness
all_valuations = np.random.random((trials, n,t))*10

for i in range(trials):
    if i % 100 == 0:
        print(i)
    items = all_items[i, :]
    valuations = all_valuations[i, :, :]
    data = (n,t, np.zeros((n,t)), items, valuations)
    
    out = run_rr2(inputs=data)
    assert check_all_efx(out[2], out[4]), "EFX SHOULD HOLD"
    while not check_all_done(out[2], out[3]):
        print('i = ', i, 'trying again...')
        break
        data = (n,t, np.zeros((n,t)), items, valuations)
        out = run_rr2(inputs=data)  # "ALL ITEMS SHOULD BE ALLOCATED"

0
100
200
i =  270 trying again...
300
400
500
600
700
800
i =  885 trying again...
900


In [369]:
out

(3,
 3,
 array([[0., 0., 8.],
        [5., 5., 0.],
        [5., 2., 2.]]),
 array([10, 10, 10]),
 array([[7.91158327, 9.02878016, 9.66212463],
        [6.49925211, 4.21902796, 2.62852868],
        [9.60887118, 0.55570066, 9.0694819 ]]))

In [373]:
from numpy import array

In [476]:
run_rr2((3,
 3,
 np.zeros((n,t)),
 array([10, 8, 10]),
 array([[7.91158327, 9.02878016, 9.66212463],
        [6.49925211, 4.21902796, 2.62852868],
        [9.60887118, 0.55570066, 9.0694819 ]])), random=True)

randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing
randomizing


(3,
 3,
 array([[0., 0., 8.],
        [5., 5., 0.],
        [5., 2., 2.]]),
 array([10,  8, 10]),
 array([[7.91158327, 9.02878016, 9.66212463],
        [6.49925211, 4.21902796, 2.62852868],
        [9.60887118, 0.55570066, 9.0694819 ]]))

In [423]:
successes

array([1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 0.7, 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ,
       1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1.

Could prove: if EFX exists, there's an EFX solution that is reachable by the algorithm.

That is, at most $3^t$ bundles. Any two players with same preference order have same number of favorite, next favorite, ..., until difference of exactly 1.

EFX exists, means there is a setup B_1, ..., B_N so that union of B_i is all m goods. and B_1, to B_N are in EFX. 

Can we adjust items and always improve a measure that gets there?

what about when t=2? then there is a b_1, ..., b_n... that is in EFX, that's in the convex hull of points, or so...
Can we always reduce them inwards? Can we do it with some small number of charity?

Really, if EFX exists everywhere, will it exist everywhere with 3^t bundles?